## 헌법 전문을 OpenSearch VectorSotre에 저장

### OpenSearch Client 생성

### **순서**
#### on WSL2 Ubuntu 22.02
- opensearch 설치  
    - default install drectory: /usr/share/opensearch  
- opensearch-dashboards 설치  
- opensearch-python 설치 (pip install opensearch-py)  
- nori plugin 설치  
    ```
    cd /usr/share
    opensearch/bin/opensearch-plugin install analysis-nori  
    ```
- nori plugin 설치 확인 (opensearch/bin/opensearch-plugin list)
- opensearch 실행 (sudo systemctl start opensearch)
- opensearch-dashboards 실행 (sudo systemctl start opensearch-dashboards)
- opensearch 실행 확인 (curl -XGET https://localhost:9200)
- opensearch-dashboards 실행 확인 (curl -XGET https://localhost:5601)
- opensearch-dashboards 접속 (https://localhost:5601)
- 분석할 문서를 PDFLoader load
- 문서 Chunking
- OpenSearchVectorSearch를 이용하여 index 생성
- 자동 생성된 인덱스 확인
- nori 형태소 분석기를 이용하기 위해 index 생성 및 재인덱싱

### Bedrock 연결 확인 및 OpenSearch 연결 확인

####  Bedrock 연결 구성 및 확인

In [12]:
# crate boto3 session
import boto3
session = boto3.Session(region_name='us-east-1')

# create bedrock client
br_client = session.client('bedrock-runtime')

In [6]:
# create a embeding model
from langchain.embeddings import BedrockEmbeddings

llm_emb = BedrockEmbeddings(
    model_id='amazon.titan-embed-text-v1',
    client=br_client,
    region_name='us-east-1'
)
llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f3d9666e990>, region_name='us-east-1', credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

#### OpenSearch 연결확인

In [2]:

from opensearchpy import OpenSearch

#  opensearch info
host = 'localhost'
port = 9200
http_auth = ('admin', 'admin')
opensearch_endpoint = f'https://{host}:{port}'

# certs info: https://www.notion.so/Certificate-Password-8087893ff1b24dd0b71e66c9916fa55a
ca_cert = 'certs/root-ca.pem'
client_cert = 'certs/admin.pem'
client_key = 'certs/admin-key.pem'


# create client
os_client = OpenSearch(
    hosts=[opensearch_endpoint],
    http_auth=http_auth,
    ca_certs=ca_cert,
    client_cert=client_cert,
    client_key=client_key,
    use_ssl=True,
    verify_certs=True,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

# verify connection
print(os_client.info())


{'name': 'DESKTOP-WHPARK', 'cluster_name': 'opensearch', 'cluster_uuid': 'Fu1wEBfcT_OjptkNYtB1WA', 'version': {'distribution': 'opensearch', 'number': '2.11.1', 'build_type': 'deb', 'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752', 'build_date': '2023-11-29T21:43:44.221253956Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


### 문서 임베딩 및 Opensearch 인덱싱

#### 문서 로딩 및 page split

In [9]:
from langchain.document_loaders import PyPDFLoader

# create a document loader
loader = PyPDFLoader("data/Constitution.pdf")
# split document into pages
documents = loader.load_and_split()
# print first page
print(documents[0])

page_content='대한민국헌법\n 법제처 - 1 / 11 -\n  국가법령정보센터대한민국헌법\n[시행 1988.2.25.] [헌법 제10호, 1987.10.29., 전부개정 ]\n        유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 \n항거한 4·19 민주이념을 계승하고 , 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로\n써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며 , 자율과 조화를 바탕으로 자유민주적 기\n본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 \n최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향\n상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유\n와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회\n의 의결을 거쳐 국민투표에 의하여 개정한다 . \n       제1장 총강 \n제1조 ①대한민국은 민주공화국이다 .\n  ②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다 .\n제2조 ①대한민국의 국민이 되는 요건은 법률로 정한다 .\n  ②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.\n제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.\n제4조 대한민국은 통일을 지향하며 , 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다 .\n제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다 .\n  ②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다 .\n제6조 ①헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다 .\n  ②외국인은 국제법과 조약이 정

#### Text Split

In [19]:
# token 길이 확인 function 생성
import tiktoken
# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def tiktoken_length(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    return len(tokens)


# create a recursive text splitter
# https://python.langchain.com/docs/modules/data_connection/document_transformers/#get-started-with-text-splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128,
    separators=["\n\n", "\n", "."," ", ""],
    length_function=tiktoken_length
)

docs = text_splitter.split_documents(documents)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=25


#### 기존 인덱스 유무 확인

In [33]:
index_name = "constitution"

index_exists = os_client.indices.exists(index=index_name)
if index_exists:
    os_client.indices.delete(index=index_name)
    print(f"Deleted index {index_name}")
else:
    print(f"Index {index_name} does not exist")


Deleted index constitution


#### OpenSearch Index 생성

In [39]:
index_name = "constitution-with-nori"
index_body = {
    "settings": {
        "index": {
            "number_of_shards": 5,
            "number_of_replicas": 1,
            "knn": "true",
            "knn.algo_param": { "ef_search": 512}
        },
        "analysis": {
            "tokenizer": {
                "nori": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "discard_punctuation": "true"
                }
            },
            "analyzer": {
                "nori_analyzer": {
                    "type": "custom",
                    "char_filter": ["html_strip"],
                    "tokenizer": "nori",
                    "filter": ["nori_number", "nori_readingform", "lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                },
                "analyzer": "nori_analyzer",
                "search_analyzer": "nori_analyzer"
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {
                        "m": 16,
                        "ef_construction": 512
                    }
                }
            }
        }
    }
}

os_client.indices.create(index=index_name, body=index_body)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'constitution-with-nori'}

In [40]:
# langchain의 OpenSerachVectorSearch를 이용하여 index 생성
# https://python.langchain.com/docs/integrations/vectorstores/opensearch

# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
from langchain.vectorstores import OpenSearchVectorSearch

docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    llm_emb,
    index_name=index_name,
    opensearch_url=opensearch_endpoint,
    http_auth=http_auth,
    ca_certs=ca_cert,
    client_cert=client_cert,
    client_key=client_key,
    use_ssl=True,
    verify_certs=True,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)



In [41]:
query="민주공화국"
docs = docsearch.similarity_search(query=query, k=3)
print(docs[0].page_content)

대한민국헌법
 법제처 - 1 / 11 -
  국가법령정보센터대한민국헌법
[시행 1988.2.25.] [헌법 제10호, 1987.10.29., 전부개정 ]
        유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 
항거한 4·19 민주이념을 계승하고 , 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로
써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며 , 자율과 조화를 바탕으로 자유민주적 기
본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 
최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향
상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유
와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회
의 의결을 거쳐 국민투표에 의하여 개정한다 . 
       제1장 총강 
제1조 ①대한민국은 민주공화국이다 .
  ②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다 .
제2조 ①대한민국의 국민이 되는 요건은 법률로 정한다 .
  ②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.
제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.
제4조 대한민국은 통일을 지향하며 , 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다 .
제5조 ①대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다 .
  ②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다 .
제6조 ①헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다 .
